# Отбор зрителей канала
Пример формирования целевой аудитории на основе просмотра телеканала и ее использование в расчетах

## Описание задачи и условий расчета
- Регион: Россия 0+
- Период: 25 декабря 2022 года
- Временной интервал: 19:00-23:00
- ЦА: все 14-44, смотревшие телеканал ТНТ 24.12.2022 года с 19:00 до 23:00
- Место просмотра: все места (дом и дача)
- Каналы: все каналы проекта TV Index
- Статистики: AvReach%

## Инициализация

При построении отчета первый шаг в любом ноутбуке - загрузка библиотек, которые помогут обращаться к TVI API и работать с данными.

Выполните следующую ячейку, для этого перейдите в нее и нажмите Ctrl+Enter

In [ ]:
%reload_ext autoreload
%autoreload 2

import pandas as pd
import json

from mediascope_api.core import net as mscore
from mediascope_api.mediavortex import tasks as cwt
from mediascope_api.mediavortex import catalogs as cwc

# Настраиваем отображение

# Включаем отображение всех колонок
pd.set_option('display.max_columns', None)
# Задаем максимальное количество выводимых строк. Раскомментируйте нужную строку
# 200 строк
# pd.set_option("display.max_rows", 200)
# Отображаем все строки. ВАЖНО! Отображение большого DataFrame требует много ресурсов
# pd.set_option("display.max_rows", None)

# Cоздаем объекты для работы с TVI API
mnet = mscore.MediascopeApiNetwork()
mtask = cwt.MediaVortexTask()
cats = cwc.MediaVortexCats()

## Справочники

Получим идентификаторы, которые будут использоваться для формирования условий расчета

In [ ]:
# В справочнике телекомпаний получим id ТНТ
cats.get_tv_company(name=["ТНТ (СЕТЕВОЕ ВЕЩАНИЕ)"])

## Создание целевой аудитории
Отберем респондентов на основе их телепросмотра и сохраним в переменную. Для этого посчитаем специальный отчет Consumption Target.

### Формирование задания
На этом этапе зададим все характеристики аудитории: возраст 14-44, смотрели канал ТНТ 24.12.2022 года с 19:00 до 23:00.

In [ ]:
# Период указывается в виде списка ('Начало', 'Конец') в формате 'YYYY-MM-DD'. 
date_filter = [('2022-12-24', '2022-12-24')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем временной интервал: с 19:00 до 23:00
time_filter = 'timeBand1 >= 190000 AND timeBand1 < 230000'

# Задаем ЦА: Все 14-44
basedemo_filter = 'age >= 14 and age <= 44'

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter = None

# Задаем каналы: ТНТ
company_filter = 'tvCompanyId IN (1870)'

# Фильтр программ
program_filter = None

# Фильтр блоков
break_filter = None

# Фильтр роликов
ad_filter = None

# Указываем срезы
slices = []

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all 
}

Статистики в отчете Consumption Target представляют собой тип просмотра, по которому отбираются зрители:
- CommonWatchers: просто зрители, смотрели телеканал хотя бы 1 минуту за отчетный период
- CommonNotWatchers: не зрители, не смотрели канал за отчетный период
- NGroupResp: зрители сортируются по длительности просмотра и разбиваются на n равных по количеству респондентов групп
- NGroupDur: зрители сортируются по длительности просмотра и разбиваются на n групп. 
        
Так как нам нужны просто зрители, то передадим в параметр statistics "CommonWatchers"

In [ ]:
# Указываем список статистик для расчета
statistics = ["CommonWatchers"]

Дополнительные настройки - consumption_target_options. Здесь можно указать:

- "groupsCount" - количество групп, на которое необходимо поделить отобранных респондентов. Работает в связке со статистиками NGroupResp и NGroupDur
- "intervalConditions" - длительность просмотра (в минутах). Можно указывать без конечной или начальной границы.
- "viewingType" - тип просмотра
    - "NONCONSECUTIVE" - непоследовательный просмотр, с переключениями
    - "CONSECUTIVE" - последовательный просмотр (без переключений)

Пример:

`
consumption_target_options = {
    "groupsCount": "5",
    "intervalConditions": {
        "intervals": [
            {
                "from":"5",
                "to": "100"
            }
        ],
    "viewingType": "CONSECUTIVE"
    }
}
`

В нашем случае дополнительные настройки не требуются, поэтому передадим в consumption_target_options значение `None`:

In [ ]:
consumption_target_options = None

### Расчет задания

In [ ]:
# Формируем задание для API TV Index в формате JSON
task_json = mtask.build_consumption_target_task(date_filter=date_filter, weekday_filter=weekday_filter,
                                                daytype_filter=daytype_filter, time_filter=time_filter,
                                                basedemo_filter=basedemo_filter, targetdemo_filter=targetdemo_filter, 
                                                location_filter=location_filter, company_filter=company_filter,
                                                program_filter=program_filter, break_filter=break_filter, ad_filter=ad_filter,
                                                slices=slices, statistics=statistics, options=options,
                                                consumption_target_options=consumption_target_options)

# Отправляем задание на расчет и ждем выполнения
task_consumption_target = mtask.wait_task(mtask.send_consumption_target_task(task_json))

# Получаем результат
df_cons = mtask.result2table(mtask.get_result(task_consumption_target))
df_cons

### Сохранение переменной

Для последующего использования в расчетах, сохраним полученную группу как переменную.  

In [ ]:
cats.add_custom_respondent_variable(
    resp=df_cons, #датафрейм с посчитанной переменной
    name="All14-44_TNT_20221224_19-23", #название переменной    
    is_public=False #Переменная будет доступна только для создавшего ее логина
)

In [ ]:
# выведем список всех переменных
resp_vars = cats.get_custom_respondent_variable()
resp_vars

In [ ]:
# выведем созданную нами переменную
resp_vars[resp_vars['name'] == "All14-44_TNT_20221224_19-23"]

In [ ]:
# удаление переменной (раскоментируйте строки ниже для выполнения)
#var_id = resp_vars[resp_vars['name'] == "All14-44_TNT_20221224_19-23"]['id'][0] #получаем id переменной по ее имени
#cats.delete_custom_respondent_variable(var_id) #удаляем переменную из списка

## Расчет отчета по группе зрителей
Используем сохраненную переменную для расчета timeband отчета
### Формирование задания
Зададим условия расчета

In [ ]:
# Период указывается в виде списка ('Начало', 'Конец') в формате 'YYYY-MM-DD'. 
date_filter = [('2022-12-25', '2022-12-25')]

# Задаем дни недели
weekday_filter = None

# Задаем тип дня
daytype_filter = None

# Задаем временной интервал: с 19:00 до 23:00
time_filter = 'timeBand1 >= 190000 AND timeBand1 < 230000'

# Задаем ЦА: оставляем незаполненным. Группу зрителей укажем далее в специальном параметре.
basedemo_filter = None

# Доп фильтр ЦА, нужен только в случае расчета отношения между ЦА, например, при расчете Affinity Index
targetdemo_filter = None

# Задаем место просмотра
location_filter = None

# Задаем каналы
company_filter = None

# Указываем список статистик для расчета
statistics = ['AvReachPer']

# Указываем срезы: телесеть
slices = ['tvNetName']

# Задаем условия сортировки: телесеть (от а до я)
sortings = {"tvNetName":"ASC"}

# Задаем опции расчета
options = {
    "kitId": 1, #TV Index Russia all
    "useNbd": False, #расчет охватов без nbd коррекции
}

# Задаем группу зрителей
#По имени переменной получаем ее id
var_id = resp_vars[resp_vars['name'] == "All14-44_TNT_20221224_19-23"]['id'][0] 

#id передаем в параметр custom_respondent_variable_id
custom_respondent_variable_id = var_id

In [ ]:
# Формируем задание для API TV Index в формате JSON
task_json = mtask.build_timeband_task(date_filter=date_filter, weekday_filter=weekday_filter, 
                                      daytype_filter=daytype_filter, company_filter=company_filter, 
                                      time_filter=time_filter, basedemo_filter=basedemo_filter, 
                                      targetdemo_filter=targetdemo_filter,location_filter=location_filter, 
                                      slices=slices, statistics=statistics, sortings=sortings, options=options,
                                     custom_respondent_variable_id=custom_respondent_variable_id)

# Отправляем задание на расчет и ждем выполнения
task_timeband = mtask.wait_task(mtask.send_timeband_task(task_json))

# Получаем результат
df = mtask.result2table(mtask.get_result(task_timeband))
df

## Экспорт в Excel
По умолчанию файл сохраняется в папку `excel`

In [ ]:
df_info = mtask.task_builder.get_report_info()

with pd.ExcelWriter(mtask.task_builder.get_excel_filename('01_consumption_target_simple_viewers')) as writer:
    df.to_excel(writer, 'Report', index=True)
    df_info.to_excel(writer, 'Info', index=False)